# huggingface-klue
*21-07-28 20:00*

## 1. init
- install transformers

In [25]:
#!pip install transformers
#!pip install wget

In [26]:
import wget

# YNAT 기준
# 출처 (https://github.com/KLUE-benchmark/KLUE/tree/main/klue_benchmark/ynat-v1.1)
wget.download("https://raw.githubusercontent.com/KLUE-benchmark/KLUE/main/klue_benchmark/ynat-v1.1/ynat-v1.1_train.json")
wget.download("https://raw.githubusercontent.com/KLUE-benchmark/KLUE/main/klue_benchmark/ynat-v1.1/ynat-v1.1_dev.json")

100% [..........................................................................] 9219447 / 9219447

'ynat-v1.1_dev.json'

In [36]:
from transformers import AutoTokenizer, BertForPreTraining


# 출처 (https://huggingface.co/klue/bert-base/tree/main)

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
model = BertForPreTraining.from_pretrained("klue/bert-base")

# device = torch.device("cuda")
# model = BertForPreTraining.from_pretrained("klue/bert-base").to(device)

* 오타 이슈 발생
    * 참고 자료 [링크](https://huggingface.co/transformers/model_doc/bert.html#bertforpretraining)
    * 따라서 hugging face 의 [trainsformers](https://huggingface.co/klue/bert-base/tree/main) 에서 `BertForPretraining` -> `BertForPreTraining`


## 2. data set

In [30]:
import json

with open("ynat-v1.1_train.json", 'rt',  encoding='UTF8') as f:
    train_dataset = json.load(f)
with open("ynat-v1.1_dev.json", 'rt',  encoding='UTF8') as f:
    dev_dataset = json.load(f)

In [54]:
train_dataset[1]

{'guid': 'ynat-v1_train_00001',
 'title': '어버이날 맑다가 흐려져…남부지방 옅은 황사',
 'predefined_news_category': '생활문화',
 'label': '생활문화',
 'annotations': {'annotators': ['14', '13', '07'],
  'annotations': {'first-scope': ['생활문화', '생활문화', '생활문화'],
   'second-scope': ['해당없음', '해당없음', '해당없음'],
   'third-scope': ['해당없음', '해당없음', '해당없음']}},
 'url': 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=103&sid2=248&oid=001&aid=0008384783',
 'date': '2016.05.08. 오전 5:25'}

In [80]:
# to df
id = []
title = []
label = []
for i in range(len(train_dataset)):
    id.append(train_dataset[i]["guid"][-5:])
    title.append(train_dataset[i]["title"])
    label.append(train_dataset[i]["label"])
    
train=pd.DataFrame({"id":id,
                    "title":title,
                    "label":label})
train

,id,title,label
0,00000,유튜브 내달 2일까지 크리에이터 지원 공간 운영,생활문화
1,00001,어버이날 맑다가 흐려져…남부지방 옅은 황사,생활문화
2,00002,내년부터 국가RD 평가 때 논문건수는 반영 않는다,사회
3,00003,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것,사회
4,00004,회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간,생활문화
...,...,...,...
45673,45673,사우디 아람코 주요 석유시설·유전 드론 공격에 화재,세계
45674,45674,박기원 감독 눈치 보지 말고…비예나 눈치 본 건 아닌데,스포츠
45675,45675,아시안게임 첫 경기 앞둔 라건아 인도네시아 팬이라도 ...,스포츠
45676,45676,트럼프 미중 무역협상 1단계 합의 서명식 가질 것종합,세계


In [96]:
inputs = tokenizer("안녕하세요", return_tensors='pt',truncation=True, padding=True)

## 2. Model

In [42]:
import pandas as pd
import torch
from torch.nn import functional as F
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW

In [38]:
# model

BertForPreTraining(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

In [97]:
model(inputs['input_ids'])

BertForPreTrainingOutput(loss=None, prediction_logits=tensor([[[ -5.4762,   6.2204,  -5.0630,  ...,  -6.6880,  -4.5765,  -6.6216],
         [ -6.8640,   6.1050,  -7.6171,  ..., -10.5715,  -7.0743,  -7.4227],
         [-10.7195,   5.4203,  -5.6494,  ..., -12.4277,  -8.0775,  -9.7798],
         [ -6.1298,   9.3065,  -8.6591,  ...,  -6.6128,  -4.3897,  -5.9199],
         [ -6.5147,  10.2238,  -5.4591,  ...,  -7.8623,  -5.8059,  -5.2569]]],
       grad_fn=<AddBackward0>), seq_relationship_logits=tensor([[1.4953, 0.5679]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)

## 3. Learn

In [98]:
# 파라미터를 조정하거나 구조를 변경하는 등 기타의 작업이 수행될 수 있음